In [363]:
FORM_MAPS = {
    "le génitif": "genitive",
    "gen.": "genitive",
    "gén.": "genitive",
    "gén": "genitive",
    "dat.": "dative",
    "dat": "dative",
    "plur.": "plural",
    "plur": "plural",
    "futur": "future",
    "fut.": "future",
    "participe": "participle",
    "comp.": "comparative",
    "comp": "comparative",
    "impér.": "imperative",
    "impér": "imperative"
}

In [408]:
import json
with open("sjoestedt_phonetique.json") as inf:
    DATA = json.load(inf)

In [349]:
section = 30

if not section in DATA:
    DATA[section] = []

In [384]:
DATA[section] = []

In [364]:
import re
REGEX_CASE = rf"^([^(]+) \(([^)]+)\) « ([^»]+) », ({'|'.join(FORM_MAPS.keys())}) ([^(]+) \(([^)]+)\)$"
REGEX_CASE_OF = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) « ([^»]+) »$"
REGEX_CASE_OF_BOTH = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_BASE = r"^([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_JUST_FRENCH = r"^([^(]+) « ([^»]+) »$"
REGEX_FROM = r"^([^(]+) \(([^)]+)\) « ([^»]+) »,? de ([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_IN = r"^([^(]+) \(([^)]+)\), dans ([^«]+) « ([^»]+) »$"
REGEX_OR = r"^([^(]+) \(([^)]+)\) ou ([^(]+) \(([^)]+)\) « ([^»]+) »$"

def extract(text, splitter, counter):
    text = text.replace("\u00ad", "").replace("\n", " ")
    for p in text.split(splitter):
            m = re.match(REGEX_BASE, p.strip())
            mjf = re.match(REGEX_JUST_FRENCH, p.strip())
            mf = re.match(REGEX_FROM, p.strip())
            min = re.match(REGEX_IN, p.strip())
            mc = re.match(REGEX_CASE, p.strip())
            mcofb = re.match(REGEX_CASE_OF_BOTH, p.strip())
            mcof = re.match(REGEX_CASE_OF, p.strip())
            mor = re.match(REGEX_OR, p.strip())
            if m:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": m.group(1),
                    "irish": m.group(2),
                    "french": m.group(3),
                })
                counter += 1
            elif mjf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mjf.group(1),
                    "french": mjf.group(2),
                })
                counter += 1
            elif mf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(1),
                    "irish": mf.group(2),
                    "french": mf.group(3).replace("\xad", ""),
                    "from": mf.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(4),
                    "irish": mf.group(5),
                    "french": mf.group(6).replace("\xad", ""),
                })
                counter += 1
            elif mor:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(1),
                    "irish": mor.group(2),
                    "french": mor.group(5),
                    "alt": mor.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(3),
                    "irish": mor.group(4),
                    "french": mor.group(5),
                    "alt": mor.group(2)
                })
                counter += 1
            elif min:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(1),
                    "irish": min.group(2),
                    "in": min.group(3)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(3),
                    "french": min.group(4),
                })
                counter += 1
            elif mc:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(1),
                    "irish": mc.group(2),
                    "french": mc.group(3),
                    FORM_MAPS[mc.group(4)]: mc.group(6)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(5),
                    "irish": mc.group(6),
                    f"{FORM_MAPS[mc.group(4)]}_of": mc.group(2)
                })
                counter += 1
            elif mcofb:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcofb.group(1),
                    "irish": mcofb.group(2),
                    f"{FORM_MAPS[mcofb.group(3)]}_of": mcofb.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "irish": mcofb.group(5),
                    "transcription": mcofb.group(4),
                    "french": mcofb.group(6),
                    FORM_MAPS[mcofb.group(3)]: mcofb.group(2)
                })
                counter += 1
            elif mcof:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(1),
                    "irish": mcof.group(2),
                    f"{FORM_MAPS[mcof.group(3)]}_of": mcof.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(4),
                    "french": mcof.group(5),
                    FORM_MAPS[mcof.group(3)]: mcof.group(2)
                })
                counter += 1
            else:
                print(p)
    print(counter)


§ 39. Exemples de gʺ :

gʺrʹaᴜn (greann) « affection » ; gʺrʹëimʹ (greim) « prise » ; gʺrʹαs (greas) « quantité, ce que l’on peut faire, prendre, etc., en une fois » ; gʺlʹe꞉ (glé) « beau, brillant » ; gʺlʹïmaꞏχ (gliomach) « langouste » ; gʺnʹe꞉ (gné) « éclat » ; gʺnʹĩ꞉ᵊv (gníomh) « action ».

§ 40.

sg̬ədɑ̃꞉n (scadán) « hareng » ; sg̬ʷɪ꞉lʹɩmʹ (scaoilim) « je délie » ; sg̬o꞉rnəχ (scórnach) « gorge » ; sg̬rɑ꞉bɑ̃꞉n (scrábán) « objet grossier, camelote » ; sg̬lɑ꞉vʷɪ꞉ᵊχt (sclábhui­dheacht) « peiner cruelle­ment ».

αsg̬ʷɩnʹɩ (eascaine) « juron » ; ɔsg̬ʷɩlʹtʹ (oscailt) « ouvrir » ; lɑsg̬ə (lasca) « empeigne ».

ì꞉asg̬ (éasc) « défaut » ; lʹαsg̬ (leasc) « mou, paresseux » ; plᴇ̈꞉ᵊsg̬ (plaosc) « crâne ».

§ 41.
ŋ (écrit ‑ng‑).
ŋ s’articule comme g, mais est fortement nasalisé sur toute sa durée. La vibration nasale commence avant le début de l’implosion (particu­lière­ment quand ŋ est implosif) et se prolonge après l’explosion (particu­lière­ment quand ŋ est explosif). Les voyelles précé­dentes et suivantes sont donc partielle­ment nasa­lisées.

La tension est moindre que pour g.

ŋ se rencontre en contact avec des voyelles ou avec des consonnes non palatales.

A l’initiale, ŋ ne se rencontre que comme alter­nance gram­maticale de g. En position médiane et finale, ŋ est d’ordinaire appuyé sur une gutturale vélaire suivante, quoique la pronon­ciation ŋ, au lieu de ŋg, plus usuel, se rencontre en position médiane.

A l’initiale devant rʹ, lʹ, nʹ, apparaît (comme alter­nance gram­maticale de gʺ) une variété inter­médiaire entre ŋ et ŋʹ, que l’on peut noter ŋʺ.

sə ŋʌrt (insan ngort) « dans le pré » ; ə ŋlɑkfɑ꞉ eꞏ (an nglacfá é ?) « est-ce que tu le prendrais » tɑ꞉ⁱmʹ bᴀᴜr ɛg ə ŋɪ̃꞉h (táim bodhar ag an ngaoith) « le vent m’assourdit ».

kʲɑ̃ŋgəl (ceangal) « lien » (aussi kʲɑ̃ŋəl) ; kᴜ꞉ŋkəs (conncas) « taqui­nerie » ; frᴀᴜŋkəχ (franncach) « français » ; ʃrʹaᴜŋgɑ̃꞉n (sreanngán) « brouil­lard épais » (aussi ʃrʹaᴜ̃ŋɑ̃꞉n) ; grᴜ̃꞉ŋgə (gronnga) « apparence congelée » (aussi grᴜ̃꞉ŋə) ; stᴀᴜ̃ŋkəhə (stann­caithe) « ratatiné » ; ᴜ̃꞉ŋgə (ionga) « ongle » (aussi ᴜ꞉ŋə).

kᴜ̃꞉ŋg (cumhang) « étroit » ; fʹrʹaᴜŋg (freanng) « crampe, cour­bature » ; lᴜ̃꞉ŋg (lung) « navire » ; gʲaᴜŋk (geannc) « nez en pied de marmite », d’où « grimace ».

§ 42. Exemples de ŋʺ :

ɛg ə ŋʺrʹe꞉nʹ ou ɛg ə ŋʺrʹì꞉an (ag an ngrian) « par le soleil » ; ɛs ə ŋʺrʹe꞉gʹɩʃ (as an ngréigis) « (traduire) du grec ».

ə ŋ″lʹaᴜn e꞉gʹɩnʹtʹ (i ngleann éigin) « dans une certaine vallée ».

Tous mots que nous notons ordinaire­ment par ŋ : ɛg ə ŋrʹe꞉nʹ, etc., etc.

§ 44.
kʹ (écrit c, et, à l’intérieur, -gf- ou -gth-).
kʹ est une occlusive palatale, sourde, forte, aspirée. Le glide j qui la suit devant voyelles d’arrière ou mixtes d’arrière est assourdi par l’aspi­ration.

kʹ se rencontre en contact avec des voyelles ou avec des consonnes palatales, excepté après s ou devant rʹ, lʹ, nʹ (cf. § 37).

kʹigʹɩlʹɩʃ (cigilis) « chatouil­le­ment » ; kʹi꞉nʹ (cinn), gén. et plur. de kʲaᴜn (ceann) « tête » ; kʲaᴜnsᴜ꞉ (ceannsú) « pacifier » ; kʹᴇlʹɩmʹ (ceilim) « je cache » ; kʹαrt (ceart) « droit » ; kʲo꞉l (céol) « musique » ; kʲo꞉ (ceó) « brouil­lard » ; kʲᴜ꞉ntəχ (cionntach) « coupable ».

ənakʹɩ (i n-aice) « près de » ; ɩkʹi (aici) « à elle » ; lʹᴇkʹəd (leigfead) « je laisserai », fut. de lʹᴇgʹɩmʹ (leigim) « je laisse » ; hikʹəχ ʃeꞏ (thuig­feadh sé) « il compren­drait » de tɪgʹɩmʹ (tuigim) « je comprends » ; tʹrʹe꞉kʹɩ (tréigthe) « passée (couleur) ».

mʹikʹ (mic), gén. de mɑk (mac) « fils » : mʷɪkʹɩ (muice), mʷɪkʹ (muic), gén. et dat. de mᴜk (muc) « porc » ; stalʹkʹ (stailc) « chose, personne immobile et comme pétrifiée ».

§ 45.
gʹ (écrit g, ou gc- à l’initiale, -gh, -dh à la finale).
gʹ s’articule comme kʹ mais est sonore, non aspiré, et se prononce avec une tension moindre.

gʹ se rencontre dans les mêmes condi­tions que kʹ.

gʹi꞉ᵊtɑ꞉ⁱlʹ (gíotáil) « travail léger » ; gʹì꞉ag (géag) « bras » ; gʹe꞉mʹ (géim) « rugisse­ment » ; gʹᴇtʹ (geit) « sursaut de frayeur » ; gʹïrə (giorra), comp. de gʲɑ꞉r (gearr) « rapproché » ; gʹαl (geal) « brillant » ; gʲo꞉kɑ̃꞉n (geócán) « flûte » ; gʲo꞉ⁱnʹ (geóin) « tumulte, rumeur » ; gʲλks (giucs) « mot, syllabe » ; ɛr ɩ gʲaᴜn (ar an gceann) « sur la tête » ; sɩ gʲaᴜntər so (insan gceanntar sô) « dans cette région » ; ʃαχ(t) gʹi꞉nʹ (seacht gcinn) « sept unités ».

αrʹɩgʹəd (airgead) « argent » ; e꞉gʹɩnʹtʹ (éigin) « quelque» ; o꞉ⁱgʹɩ (óige) « jeunesse ».

bɑ꞉ʃtʹɩgʹ (báistigh), dat. de bɑ꞉ʃtʹəχ (báisteach) « pluie » ; krᴜꞏɛgʹ (cruaidh) « dur » ; mʹᴇrʹɩgʹ (meirg) « rouille » ; sɪgʹ (suidh) « assieds-toi » ; ᴜ꞉nʃɩgʹ (óinsigh), datif de ᴜ꞉nʃəχ (óinseach) « folle ».

§ 46.
g̬ʹ (écrit c ou g).
g̬ʹ est une forme assourdie de gʹ, qui n’apparaît qu’après s ou ʃ.

sg̬ʹì꞉al ou ʃg̬ʹì꞉al (scéal) « histoire » ; sg̬ʹiꞏən ou ʃg̬ʹiꞏən (scian) « couteau » ; sg̬ʲλbʷɩmʹ ou ʃg̬ʲλbʷɩmʹ (sciobaim) « je débar­rasse, je balaye » ; sg̬ʲλlpə ou ʃg̬ʲλlpə (sciolpa) et sg̬ʲᴜ꞉rʃɩ (sciúirse) « morceau, fragment ».

ə naʃg̬ʹɩ (i n-aisce) « gratuite­ment » ; ə daʃg̬ʹɩ (i dtaisce) « de côté, en réserve » ; iʃg̬ʹɩ (uisce) « eau » ; fɑ꞉ʃg̬ʹɩmʹ (fáiscim) « je serre » ; lʹᴇʃg̬ʹɩ (leisce) « paresse ».

e꞉ʃg̬ʹ (éisc), gén. de iꞏəsg̬ (iasc) « poisson » ; tᴜꞏɛrʹɩʃg̬ʹ (tuairisc) « nouvelle ».

§ 47
ŋʹ (écrit ng).
ŋʹ s’articule comme gʹ, mais est fortement nasalisé sur toute sa durée. La vibration nasale commence avant le début de l’implosion (surtout quand ŋʹ est implosif) et se prolonge après l’explosion (surtout quand ŋʹ est explosif). Les voyelles précé­dentes et suivantes sont donc en général nasa­lisées.

La tension est bien moindre que pour gʹ, au point que, chez certains sujets, l’occlusion a tendance à être impar­faite.

ŋʹ se rencontre en contact avec des voyelles ou avec des consonnes non vélaires.

A l’initiale ŋʹ ne se rencontre que comme alter­nance gramma­ticale de gʹ. En position médiane ou finale ŋʹ est toujours appuyée sur une gutturale palatale suivante.

sɩ ŋʹi꞉rʹɩ (insan ngeimh­readh) « durant l’hiver », de gʹi꞉rʹɩ (geimh­readh) « hiver » ; ɩ ŋʹïraχt (i ngior­racht) « à proximité » ; ɩ ŋʹαlən tᴜꞏ dɔm eꞏ (an ngeallann tu dom é ?) « est-ce que tu me le promets ? ».

daŋʹgʹən (daingean) « solide » ; lɪŋʹgʹəs (luingeas) « flotte ».

kʷɪ꞉ŋʹgʹ (cuing) « obli­gation » ; lɪ꞉ŋʹgʹ (luing) et lɪŋʹgʹɩ (luinge), dat. et gén. de lɔŋg (long) « navire » ; mʷɪ꞉ŋʹgʹ (muing) « crinière ».

In [415]:
# DATA[section] = []
section = 38
if not section in DATA:
    DATA[section] = []
TEXT = "o꞉g (óg) « jeune »"
extract(TEXT, ";", 25)

 o꞉g (óg) « jeune
25


In [417]:
tmp = DATA["38"] + DATA[38]

In [418]:
tmp

[{'section': 38,
  'id': '38_1',
  'transcription': 'gɑlər',
  'irish': 'galar',
  'french': 'maladie'},
 {'section': 38,
  'id': '38_2',
  'transcription': 'gɑ̃n',
  'irish': 'gan',
  'french': 'sans'},
 {'section': 38,
  'id': '38_3',
  'transcription': 'gɑ꞉rʹi꞉',
  'irish': 'gáiridhe',
  'french': 'rire'},
 {'section': 38,
  'id': '38_4',
  'transcription': ['gɔb', 'go̤b'],
  'irish': 'gob',
  'french': 'bec'},
 {'section': 38,
  'id': '38_5',
  'transcription': 'gʌrtʹ',
  'irish': 'guirt',
  'french': 'salé'},
 {'section': 38,
  'id': '38_6',
  'transcription': 'gᴜ꞉nə',
  'irish': 'gúna',
  'french': 'robe'},
 {'section': 38,
  'id': '38_7',
  'transcription': 'gʷᴇ̈꞉h',
  'irish': 'gaoth',
  'french': 'vent'},
 {'section': 38,
  'id': '38_8',
  'transcription': 'gʷɪ꞉',
  'irish': 'guidhe',
  'french': 'prier'},
 {'section': 38,
  'id': '38_9',
  'transcription': 'glᴜ꞉n',
  'irish': 'glún',
  'french': 'genou'},
 {'section': 38,
  'id': '38_10',
  'transcription': 'glɑs',
  'irish':

In [420]:
DATA["38"] = tmp
del(DATA[38])

In [412]:
# DATA[section][-1]["transcription"] = DATA[section][-1]["transcription"].split(" ou ")
DATA[section][-1]["french"] = [DATA[section][-1]["french"], "rage"]

In [421]:
import json

if str(section) in DATA:
    tmp = DATA[str(section)] + DATA[section]
    DATA[str(section)] = tmp
    del(DATA[section])

with open("sjoestedt_phonetique.json", "w") as outf:
    json.dump(DATA, outf)